In [2]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from collections import defaultdict
import re
import pickle
import sys
from datetime import datetime

In [9]:
with open('all_heads_links_nodups.txt', 'r', encoding='utf-8') as f:
    all_heads_links = f.read().split('\n')
info_all = []
for line in all_heads_links:
    line = line.split('\t')
    info_all.append((line[0], line[1]))

with open('polemic_headlines_by_themes.txt', 'r', encoding='utf-8') as f1:
    topics = f1.read().split('\n\n')
cluster_heads = []
for group in topics:
    cluster_heads.append(group.split('\n'))

to_get_comments = []
for cluster in cluster_heads:
    clstr_arr = []
    for headline in cluster[1:]:
        for head in info_all:
            if headline == head[0]:
                clstr_arr.append(head[1])
    clstr_arr = list(set(clstr_arr))
    to_get_comments.append((cluster[0], clstr_arr))

In [10]:
to_get_comments[0]

('6',
 ['https://ria.ru/world/20170504/1493626174.html',
  'https://ria.ru/world/20170502/1493537580.html',
  'https://ria.ru/world/20170206/1487252689.html',
  'https://ria.ru/world/20170302/1489156170.html',
  'https://ria.ru/world/20170405/1491538862.html',
  'https://ria.ru/world/20170423/1492892954.html',
  'https://ria.ru/politics/20170518/1494587195.html',
  'https://ria.ru/world/20170507/1493859279.html',
  'https://ria.ru/world/20170506/1493812162.html',
  'https://ria.ru/world/20170501/1493432663.html',
  'https://ria.ru/world/20170420/1492698853.html',
  'https://ria.ru/world/20170508/1493893247.html',
  'https://ria.ru/world/20170612/1496326049.html',
  'https://ria.ru/world/20170316/1490150792.html',
  'https://ria.ru/world/20170305/1489320492.html',
  'https://ria.ru/world/20170423/1492915883.html',
  'https://ria.ru/world/20170423/1492921318.html',
  'https://ria.ru/world/20170428/1493313037.html'])

In [29]:
def get_data_id(comment_sect, data_id):
    '''
    data_id последнего комента верхнего уровня с текущей страницы для ссылки на следующую страницу
    '''      
    for bla in comment_sect:
        try:
            data_id = bla['data-ts']
        except KeyError:
            pass
        return data_id

def get_text(text, comment):
    '''
    Достать текст из тега
    '''
    for sect in text:
        comment += str(sect)
    comment = re.sub('<br/>', '', comment)
    return comment

def get_author(comment_sect, users):
    '''
    собрать комментировавших и отвечавших юзеров для каждого коммента и сколько раз каждый из них писал
    '''
    for author in comment_sect.findAll('a', {'class':'b-comments__item-author'}):
        if author:
            for each in author:
                for nick in each:
                    users[nick] += 1

def get_comment_text(comment_sect, each_comment):
    '''
    сбор непосредственно комментов и их дат
    '''
    for comment_itself in comment_sect.find('div', {'class':'b-comments__item m-first-level'}):
        if comment_itself:
            comment = ''
            date = comment_itself.find('div', {'class':'b-comments__item-date'})
            if date:
                date = date['datetime']
                text = comment_itself.find('div', {'class':'b-comments__item-text'})
                if text:
                    each_comment.append(get_text(text, comment))
    return date

def get_replies(comment_sect, each_comment, users):
    '''
    сбор ответов на комменты
    '''
    reply = ''
    reply_date = 0
    for reply_itself in comment_sect.find('div', {'class':'b-comments__list-answer'}):#'b-comments__list-items'
        if reply_itself:
            for for_reply_dates in reply_itself.findAll('div', {'class':'b-comments__item'}):
                if for_reply_dates:
                    reply_date = for_reply_dates['data-ts']
            text = reply_itself.findAll('div', {'class':'b-comments__item-text'})
            #date последнего ответа на коммент
            if text:                
                for sect in text:
                    each_comment.append(get_text(sect, reply))
                        
            next_reply_page = comment_sect.find('div', {'class':'b-comments__list-answer-load'})
            if next_reply_page:
                try:
                    next_reply_page['style']
                except KeyError:
                    next_replies(next_reply_page, users, reply_date, each_comment)

def next_replies(next_reply_page, users, reply_date, each_comment):
    '''
    перейти на страницу со всеми ответами, выкачать их и логины юзеров
    '''
    next_reply = ''
    parent_id = next_reply_page['data-parent_id']
    reply_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&parent_id='+parent_id+'&date='+str(reply_date)
    req_reply = requests.get(reply_link)
    soup_reply = BeautifulSoup(req_reply.text, 'lxml')
    
    for by_reply in soup_reply.findAll('div', {'class':'b-comments__item'}):
        if by_reply:
            reply_date = by_reply['data-ts']
            get_author(by_reply, users)
            text = by_reply.findAll('div', {'class':'b-comments__item-text'})
            if text:
                for sect in text:
                    each_comment.append(get_text(sect, next_reply))
            next_reply_page = by_reply.find('div', {'class':'b-comments__list-answer-load'})
            if next_reply_page:
                print('Noooooooooooooo')
                next_replies(next_reply_page, users, reply_date)
    return next_reply_page, users
        
def main_walker(comments_link, article_id, headline_comments_replies):
    '''
    Функция получает на вход ссылку, и собирает в массив кортежи из комметария, 
    ответов на него и словаря с пользователями и количеством комментов/ответов от них
    '''
    req = requests.get(comments_link)
    soup = BeautifulSoup(req.text, 'lxml')
    data_id = 0
    for comment_sect in soup.findAll('div', {'class':'b-comments__list-tree'}):
        data_id = get_data_id(comment_sect, data_id) #data_id последнего комента верхнего уровня с текущей страницы для ссылки на следующую страницу
        date = ''
        each_comment = []
        date = get_comment_text(comment_sect, each_comment) #сбор непосредственно комментов и их дат
        users = defaultdict(int)
        get_author(comment_sect, users)
        get_replies(comment_sect, each_comment, users)
        headline_comments_replies.append((date, list(set(each_comment)), users))
        
    next_p = soup.find('div', {'class':'b-comments__list-answer'})
    if next_p:
        comments_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&sort=desc&date='+data_id
        main_walker(comments_link, article_id, headline_comments_replies)

start_time = datetime.now()
trial = ['https://ria.ru/world/20171010/1506569366.html']
polemic_comments = []
for cortege in to_get_comments:
    polemic_cortege = []
    print('Now processing cluster №\t', cortege[0])
    for headline in cortege[1]:
        link = cortege[1][headline]
        article_id = link[-15:-5] 
        comments_link = 'https://ria.ru/services/comments/get_list/?article_id='+article_id+'&sort=desc&add_class=m-main-list'
        headline_comments_replies = []
        main_walker(comments_link, article_id, headline_comments_replies)
        polemic_cortege.append((headline, cortege[1][headline], headline_comments_replies))
    polemic_comments.append((cortege[0], polemic_cortege))

sys.setrecursionlimit(100000)
with open('polemic_comments.pkl', 'wb') as fp:
    pickle.dump(polemic_comments, fp)
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))    

Now processing  6  cluster
Now processing  7  cluster
Now processing  12  cluster
Now processing  16  cluster
Now processing  20  cluster
Now processing  21  cluster
Now processing  24  cluster
Now processing  30  cluster
Now processing  33  cluster
Now processing  34  cluster
Duration: 0:06:25.877341
